## Importando as bibliotecas

In [10]:
import requests
from bs4 import BeautifulSoup as bs
import re

## Carregando uma página

In [2]:
# Carregando uma webpage
r = requests.get('https://keithgalli.github.io/web-scraping/example.html')

#Convertendo para um objeto beautiful soup
soup = bs(r.content)

# Printando o html
print(soup.prettify()) #.prettify() = melhora a visualização

<html>
 <head>
  <title>
   HTML Example
  </title>
 </head>
 <body>
  <div align="middle">
   <h1>
    HTML Webpage
   </h1>
   <p>
    Link to more interesting example:
    <a href="https://keithgalli.github.io/web-scraping/webpage.html">
     keithgalli.github.io/web-scraping/webpage.html
    </a>
   </p>
  </div>
  <h2>
   A Header
  </h2>
  <p>
   <i>
    Some italicized text
   </i>
  </p>
  <h2>
   Another header
  </h2>
  <p id="paragraph-id">
   <b>
    Some bold text
   </b>
  </p>
 </body>
</html>



## Começando a utilzar BS para fazer o scrape

### find e find_all()

In [6]:
# Passando um elemento para a função

first_header = soup.find('h2') # Retorna a primeira tag <h2>
print(first_header)


headers = soup.find_all('h2') # Retorna uma lista com todas as tags <h2>
print(headers)

<h2>A Header</h2>
[<h2>A Header</h2>, <h2>Another header</h2>]


In [7]:
# Passando uma lista para a função

first_header = soup.find(['h2', 'h1']) # Retorna a primeira tag <h2> ou <h1> que encontrar
print(first_header)


headers = soup.find_all(['h2', 'h1']) # Retorna uma lista com todas as tags <h2> e <h1> que encontrou
print(headers)

<h1>HTML Webpage</h1>
[<h1>HTML Webpage</h1>, <h2>A Header</h2>, <h2>Another header</h2>]


In [8]:
# Passando atributos para find/find_all

paragrafo = soup.find_all('p', attrs={'id': 'paragraph-id'})
print(paragrafo)

[<p id="paragraph-id"><b>Some bold text</b></p>]


In [9]:
# Fazendo busca aninhada

body = soup.find('body') # Retorna o <body> do html
div = body.find('div') # Retorna a primeira <div> do body
header = div.find('h1') # Retorna o primeiro <h1> da div
header

<h1>HTML Webpage</h1>

In [13]:
# Procurando por strings específicas no html

paragrafo = soup.find_all('p', string=re.compile('Some')) # Retorna todos as tags <p> que contenham a string 'Some' no texto
paragrafo

headers = soup.find_all('h2', string=re.compile('(H|h)eader')) # Retorna todos as tags <h2> que contenham a string 'Header' ou 'header' no texto
headers

[<h2>A Header</h2>, <h2>Another header</h2>]

### select

Seleciona como o selector do CSS

In [15]:
content = soup.select('p') # Retorna todas as tags <p>
content

[<p>Link to more interesting example: <a href="https://keithgalli.github.io/web-scraping/webpage.html">keithgalli.github.io/web-scraping/webpage.html</a></p>,
 <p><i>Some italicized text</i></p>,
 <p id="paragraph-id"><b>Some bold text</b></p>]

In [16]:
content = soup.select('div p') # Retorna todas as tags <p> que estão dentro de uma tag <div>
content

[<p>Link to more interesting example: <a href="https://keithgalli.github.io/web-scraping/webpage.html">keithgalli.github.io/web-scraping/webpage.html</a></p>]

In [21]:
paragrafo = soup.select('h2 ~ p') # Seleciona todos as tags <p> irmãs de <h2>, que vêm depois de <h2> "irmãs <p> mais novas"
paragrafo

[<p><i>Some italicized text</i></p>,
 <p id="paragraph-id"><b>Some bold text</b></p>]

In [19]:
bold_text = soup.select('p#paragraph-id b') # Seleciona todos as tags <b> que estão dentro de uma tag <p> com id 'paragraph-id'
bold_text

[<b>Some bold text</b>]

In [24]:
paragrafo = soup.select('body > p') # Seleciona todos as tags <p> que são diretamente filhas de <body>,
print(paragrafo)

# Loop for que seleciona todas as tags <i> dentro de cada tag da lista paragrafo
for i in paragrafo:
    print(i.select('i'))

[<p><i>Some italicized text</i></p>, <p id="paragraph-id"><b>Some bold text</b></p>]
[<i>Some italicized text</i>]
[]


In [25]:
# Selecionando um elemento com uma propriedade específica
soup.select('[align=middle]')

[<div align="middle">
 <h1>HTML Webpage</h1>
 <p>Link to more interesting example: <a href="https://keithgalli.github.io/web-scraping/webpage.html">keithgalli.github.io/web-scraping/webpage.html</a></p>
 </div>]

## Get propriedades diferente do HTML

In [29]:
# .string para elementos com apenas 1 filho
header = soup.find('h2')
header.string # Retorna o texto dentro da tag

# get_text() para elementos com mais de um filho
div = soup.find('div')
print(div.prettify())
print(div.get_text()) # Retorna o texto dentro da tag

<div align="middle">
 <h1>
  HTML Webpage
 </h1>
 <p>
  Link to more interesting example:
  <a href="https://keithgalli.github.io/web-scraping/webpage.html">
   keithgalli.github.io/web-scraping/webpage.html
  </a>
 </p>
</div>


HTML Webpage
Link to more interesting example: keithgalli.github.io/web-scraping/webpage.html



In [32]:
# Pegando uma propriedade específica do elemento

link = soup.find('a')
print(link['href']) # Retorna o valor do atributo href do elemento <a>

paragrafo = soup.select('p#paragraph-id')
print(paragrafo[0]['id']) # Retorna o valor do atributo id do primeiro item da lista paragrafo

https://keithgalli.github.io/web-scraping/webpage.html
paragraph-id


## Navegação de código

In [35]:
# Path syntax

soup.body.div.h1.string # Usa a dot-notation para acessar os elemtentos do objeto soup

'HTML Webpage'

In [38]:
soup.body.find('div').find_next_siblings() # Encontra os imãos 'mais novos' da primeira tag <div>

[<h2>A Header</h2>,
 <p><i>Some italicized text</i></p>,
 <h2>Another header</h2>,
 <p id="paragraph-id"><b>Some bold text</b></p>]